In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob

import os
import sys
from pathlib import Path

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

import phys_const as pc

In [ ]:
energybinsMeV = np.array([
    1, 3, 5.2382, 8.0097, 11.442, 15.691, 20.953, 27.468,
    35.536, 45.525, 57.895, 73.212, 92.178, 115.66,
    144.74, 180.75, 225.33, 280.54
]) # MeV

In [ ]:
direct = '/home/erick/gw170817_fluid_profile/matter_1.0_ye'
nu_lib_table_dir = '/mnt/scratch/erick/jobs_emu/tables/NuLib_SFHo.h5'

In [ ]:
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=False)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Function to apply custom tick locators and other settings to an Axes object
def apply_custom_settings(ax, log_scale_y=False):

    if log_scale_y:
        # Use LogLocator for the y-axis if it's in log scale
        ax.set_yscale('log')
        ax.yaxis.set_major_locator(LogLocator(base=10.0))
        ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=100))
    else:
        # Use AutoLocator for regular scales
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Apply the AutoLocator for the x-axis
    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

# This function creates a pcolormesh plot with contour lines and saves it as a PNG file.
def plot_pcolormesh_with_contour(x, y, z, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, do_show=True, do_save=False):

    fig, ax = plt.subplots(figsize=(12, 8))

    # Normalize Z data
    min_cb_scaled = min_cb
    max_cb_scaled = max_cb

    # Plot pcolormesh
    # Set non-data points (NaN) to black in the colormap
    cmap = plt.get_cmap(colormap).copy()
    cmap.set_bad(color='black')

    # Plot pcolormesh with masked array to handle NaNs
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=cmap, vmin=min_cb_scaled, vmax=max_cb_scaled)

    # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=3)
    # ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1f')
    
    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')

    # Save figure
    if do_save:
        fig.savefig(direct+'/plots/'+filename, format='png', bbox_inches='tight')

    # Display figure
    if do_show:
        plt.show()
        # display(fig)
    
    # Close figure
    plt.close(fig)

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

# number of cells
Nx = np.array(rho_ye_T_h5py['/ncellsx'])
Ny = np.array(rho_ye_T_h5py['/ncellsy'])
Nz = np.array(rho_ye_T_h5py['/ncellsz'])
print(f'Nx = {Nx}, Ny = {Ny}, Nz = {Nz}')

# cell size
dx = ( np.array(rho_ye_T_h5py['/xmax_cm']) - np.array(rho_ye_T_h5py['/xmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsx']) # cm
dy = ( np.array(rho_ye_T_h5py['/ymax_cm']) - np.array(rho_ye_T_h5py['/ymin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsy']) # cm
dz = ( np.array(rho_ye_T_h5py['/zmax_cm']) - np.array(rho_ye_T_h5py['/zmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsz']) # cm
print(f'dx = {dx} cm, dy = {dy} cm, dz = {dz} cm')

# cell centers
xc = np.linspace(dx / 2, dx * (Nx - 0.5), Nx) # cm
yc = np.linspace(dy / 2, dy * (Ny - 0.5), Ny) # cm
zc = np.linspace(dz / 2, dz * (Nz - 0.5), Nz) # cm

# cell centers mesh
Xc, Yc, Zc = np.meshgrid(xc, yc, zc, indexing='ij') # cm

bh_radius = 5.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

distance_from_bh = np.sqrt( (Xc - bh_center_x)**2 + (Yc - bh_center_y)**2 + (Zc - bh_center_z)**2 )
mask_bh = (distance_from_bh < bh_radius)
# mask_x = (Xc < dx) | (Xc > (Nx - 1) * dx)
# mask_y = (Yc < dy) | (Yc > (Ny - 1) * dy)
# mask_z = (Zc < dz) | (Zc > (Nz - 1) * dz)
# mask = mask_bh | mask_x | mask_y | mask_z
mask = mask_bh

In [ ]:
x_slice_indx = 48 # cell number
y_slice_indx = 48 # cell number
z_slice_indx = 16 # cell number

z_slice_value = Zc[0,0,z_slice_indx] # cm
y_slice_value = Yc[0,y_slice_indx,0] # cm

# Data x and y for upper panel
xup = Xc[:, :, z_slice_indx] / 1e5 # km
yup = Yc[:, :, z_slice_indx] / 1e5 # km

# Data x and y for lower panel
xlow = Xc[:, y_slice_indx, :] / 1e5 # km
ylow = Zc[:, y_slice_indx, :] / 1e5 # km

In [ ]:
ye_slice_up   = np.array(rho_ye_T_h5py['/Ye'])       [:,:,z_slice_indx]
ye_slice_low   = np.array(rho_ye_T_h5py['/Ye'])       [:,y_slice_indx,:]

rho_slice_up  = np.array(rho_ye_T_h5py['/rho_g|ccm'])[:,:,z_slice_indx]
rho_slice_low  = np.array(rho_ye_T_h5py['/rho_g|ccm'])[:,y_slice_indx,:]

ne_slice_up = ye_slice_up * rho_slice_up / pc.PhysConst.Mp
ne_slice_low = ye_slice_low * rho_slice_low / pc.PhysConst.Mp

max_color_bar = np.max([np.max(ne_slice_low), np.max(ne_slice_up)])
min_color_bar = np.min([np.min(ne_slice_low), np.min(ne_slice_up)])

x = xup.copy()
y = yup.copy()
z = ne_slice_up.copy()
z[mask[:,:,z_slice_indx]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$y \, (\mathrm{km})$', 
                            title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$n_e$', 
                            colormap='viridis',
                            filename=f'ne_map_yx.png',
                            do_show=True,
                            do_save=True)
x = xlow.copy()
y = ylow.copy()
z = ne_slice_low.copy()
z[mask[:,y_slice_indx,:]] = np.nan

plot_pcolormesh_with_contour(x=x, 
                            y=y, 
                            z=z, 
                            min_cb=min_color_bar, 
                            max_cb=max_color_bar, 
                            x_label=r'$x \, (\mathrm{km})$', 
                            y_label=r'$z \, (\mathrm{km})$', 
                            title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$',
                            cbar_label=fr'$n_e$', 
                            colormap='viridis',
                            filename=f'ne_map_zx.png',
                            do_show=True,
                            do_save=True)

In [ ]:
# mass state 1 mass in eV [NO/IO:-sqrt(7.39e-5)]
mass1_eV = -0.008596511

# mass state 2 mass in eV (define at 0 arbitrarily because oscillations only sensitive to deltaM^2)
mass2_eV = 0

# mass state 3 mass in eV [NO:sqrt(2.449e-3) IO:-sqrt(2.509e-3)]
mass3_eV = 0.049487372

# 1-2 mixing angle in degrees [NO/IO:33.82]
theta12_degrees = 33.82
theta12_radians = np.radians(theta12_degrees)

# 2-3 mixing angle in degrees [NO:8.61 IO:8.65]
theta23_degrees = 8.61
theta23_radians = np.radians(theta23_degrees)

# 1-3 mixing angle in degrees [NO:48.3 IO:48.6]
theta13_degrees = 48.3
theta13_radians = np.radians(theta13_degrees)

# Majorana angle 1 in degrees
alpha1_degrees = 0

# Majorana angle 2 in degrees
alpha2_degrees = 0

# CP-violating phase in degrees [NO:222 IO:285]
deltaCP_degrees = 222

delta_m_square_12 = mass1_eV**2 - mass2_eV**2 # eV^2
# delta_m_square_12 *= pc.CGSUnitsConst.eV**2 # erg^2

In [ ]:
ye  = np.array(rho_ye_T_h5py['/Ye'])[:,:,:] 
rho = np.array(rho_ye_T_h5py['/rho_g|ccm'])[:,:,:] # g/ccm
ne  = ye * rho / pc.PhysConst.Mp # ccm
V_msw_erg = np.sqrt(2) * pc.PhysConst.GF * pc.PhysConst.hbarc ** 3 * ne # erg
V_msw_eV  = V_msw_erg / pc.CGSUnitsConst.eV # eV

A_cc = np.zeros((len(energybinsMeV),) + V_msw_eV.shape)  # eV^2
delta_m_square_MSW = np.zeros_like(A_cc)  # eV^2
sin_2theta_MSW = np.zeros_like(A_cc)  # dimensionless
L_osc_MSW = np.zeros_like(A_cc)  # cm

for i, E in enumerate(energybinsMeV):
    hbarc_eV_cm = pc.PhysConst.hbarc / pc.CGSUnitsConst.eV
    E_eV = E * 1e9 # eV
    A_cc[i] = 2.0 * V_msw_eV * E_eV # eV^2
    delta_m_square_MSW[i] = np.sqrt( ( delta_m_square_12 * np.cos(  2.0 * theta12_radians ) - A_cc[i] ) ** 2 + ( delta_m_square_12 * np.sin(  2.0 * theta12_radians ) ) ** 2 ) # eV^2
    sin_2theta_MSW[i] = ( delta_m_square_12 / delta_m_square_MSW[i] ) * np.sin(  2.0 * theta12_radians )
    L_osc_MSW[i] = 4.0 * np.pi * hbarc_eV_cm * E_eV / delta_m_square_MSW[i] # cm

In [ ]:
# for i, E in enumerate(energybinsMeV):
for i, E in enumerate(energybinsMeV[3:4]):

    L_osc_MSW_slice_up  = L_osc_MSW[i,:,:,z_slice_indx]
    L_osc_MSW_slice_low = L_osc_MSW[i,:,y_slice_indx,:]

    x = xup.copy()
    y = yup.copy()
    z = L_osc_MSW_slice_up.copy()
    z[mask[:,:,z_slice_indx]] = np.nan

    plot_pcolormesh_with_contour(x=x, 
                                y=y, 
                                # z=z, 
                                z=np.log10(z), 
                                min_cb=None, 
                                max_cb=None,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$y \, (\mathrm{km})$', 
                                title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$, $E = {E:.2f} \,\mathrm{{MeV}}$',
                                cbar_label=fr'$\log L_{{\mathrm{{MSW}}}}\,[cm] $', 
                                colormap='viridis',
                                filename=f'ne_map_yx.png',
                                do_show=False,
                                do_save=True)
    x = xlow.copy()
    y = ylow.copy()
    z = L_osc_MSW_slice_low.copy()
    z[mask[:,y_slice_indx,:]] = np.nan

    plot_pcolormesh_with_contour(x=x, 
                                y=y, 
                                # z=z, 
                                z=np.log10(z), 
                                min_cb=None, 
                                max_cb=None,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$, $E = {E:.2f} \,\mathrm{{MeV}}$', 
                                cbar_label=fr'$\log L_{{\mathrm{{MSW}}}}\,[cm] $', 
                                colormap='viridis',
                                filename=f'ne_map_zx.png',
                                do_show=True,
                                do_save=True)

In [ ]:
# for i, E in enumerate(energybinsMeV):
for i, E in enumerate(energybinsMeV[3:4]):

    sin_square_2theta_MSW_slice_up  = sin_2theta_MSW[i,:,:,z_slice_indx] ** 2
    sin_square_2theta_MSW_slice_low = sin_2theta_MSW[i,:,y_slice_indx,:] ** 2

    x = xup.copy()
    y = yup.copy()
    z = sin_square_2theta_MSW_slice_up.copy()
    z[mask[:,:,z_slice_indx]] = np.nan

    plot_pcolormesh_with_contour(x=x, 
                                y=y, 
                                # z=z, 
                                z=np.log10(z), 
                                min_cb=None, 
                                max_cb=None,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$y \, (\mathrm{km})$', 
                                title=fr'$z={z_slice_value/1e5:.1f}\,\mathrm{{km}}$, $E = {E:.2f} \,\mathrm{{MeV}}$',
                                cbar_label=fr'$\log \sin^2 \theta_{{\mathrm{{MSW}}}}$', 
                                colormap='viridis',
                                filename=f'ne_map_yx.png',
                                do_show=False,
                                do_save=True)
    x = xlow.copy()
    y = ylow.copy()
    z = sin_square_2theta_MSW_slice_low.copy()
    z[mask[:,y_slice_indx,:]] = np.nan

    plot_pcolormesh_with_contour(x=x, 
                                y=y, 
                                # z=z, 
                                z=np.log10(z), 
                                min_cb=None, 
                                max_cb=None,
                                x_label=r'$x \, (\mathrm{km})$', 
                                y_label=r'$z \, (\mathrm{km})$', 
                                title=rf'$y={y_slice_value/1e5:.1f}\,\mathrm{{km}}$, $E = {E:.2f} \,\mathrm{{MeV}}$', 
                                cbar_label=fr'$\log \sin^2 \theta_{{\mathrm{{MSW}}}}$', 
                                colormap='viridis',
                                filename=f'ne_map_zx.png',
                                do_show=True,
                                do_save=True)